In [34]:
# Instruções basiconas de sempre.

# Lidar com dataframes.
import numpy as np
import pandas as pd

# Lidar com gráficos.
import matplotlib.pyplot as plt
import seaborn as sns

# Conjunto de módulos do bokeh
from bokeh.io import output_notebook
from bokeh.plotting import figure, show

from bokeh.models import (ColumnDataSource
                          , CDSView
                          , GroupFilter
                          , NumeralTickFormatter
                          , HoverTool
                          , CategoricalColorMapper
                          , Div
                          , HBar)

from bokeh.layouts import (row
                           , column
                           , gridplot)

from bokeh.models.widgets import Tabs, Panel

# Para lidar com datas
from datetime import date

# Lidar com preparação de dados.
from data_prep import data_prep as dp # Eu que fiz esse modulinho ("uuuuuuuuuma bosts!").

#Bibliotecas necessárias para aquisitar as informações em json em requisitando uma API Rest.
import json
import requests

pd.set_option('display.max_columns', None)  
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 200)

# DATA CLEANING

## Aquisitando os dados através de requisições a API do Brasil.IO e transformando JSON em um DataFrame

Aqui estu pegando os dados referentes a um intervalo de 1 ano (roda enquanto abs(date.today().year - nuano_atual) > 2).

Seguindo as instruções em uma questão do [Stackoverflow](https://stackoverflow.com/questions/21104592/json-to-pandas-dataframe), temos:

In [2]:
df_gastos_parlamentares = pd.DataFrame(list())

is_another_year = False
nuano_atual = date.today().year
numero = 1

# Essa variável indica o período em anos que eu desejo analisar.
# Ex.:se eu quero só o ano vigente, coloco 1. Se eu quiser desde o ano passado, coloco 2.
anos_a_analisar = 1


while not is_another_year:
    nuano_anterior = nuano_atual
    r = requests.get("https://brasil.io/api/dataset/gastos-deputados/cota_parlamentar/data?page="+str(numero))
    if r.status_code == 200:
        json_gastos_parlamentares = json.loads(r.content)
        regex_equal_symbol = json_gastos_parlamentares["next"].rfind("=")
#         print(json_gastos_parlamentares["next"][regex_equal_symbol+1:])
        nuano_atual = json_gastos_parlamentares["results"][-1]["numano"]
        df_gastos_parlamentares = pd.concat([df_gastos_parlamentares
                                             , pd.json_normalize(json_gastos_parlamentares["results"])]
                                            , axis=0
                                            , join="outer"
                                           )
    if abs(date.today().year - nuano_atual) > anos_a_analisar:
        is_another_year = True
    
    numero += 1
    
df_gastos_parlamentares.shape

(7000, 29)

## Limpeza Grosseira
Denominei limpeza grosseira como o processo de remoção de atributos com todos valores nulos ou com cardinalidade igual a 1.
Além disso, aproveitei para converter os atributos numéricos em ints e floats.

In [3]:
print(f"""Antes da limpeza grosseira: {df_gastos_parlamentares.shape}""")
print("-"*40)

# Removendo os atributos com apenas valores nulos: 
df_gastos_parlamentares.dropna(axis=1, inplace=True)

# Removendo os atributos com cardinalidade <= 1:
lista_atributos_para_remover = list(dp.cardinalidade(df_gastos_parlamentares)[dp.cardinalidade(df_gastos_parlamentares)["Cardinalidade"] <= 1]["Atributo"].values)
df_gastos_parlamentares.drop(lista_atributos_para_remover, axis=1, inplace=True)

lista_colunas_numericas = ["vlrdocumento", "vlrliquido"]

for coluna in lista_colunas_numericas:
    df_gastos_parlamentares[coluna] = pd.to_numeric(df_gastos_parlamentares[coluna])


print(f"""Após da limpeza grosseira: {df_gastos_parlamentares.shape}""")
print("-"*40)

df_gastos_parlamentares.select_dtypes(include=["float64", "float32"]).describe().T

Antes da limpeza grosseira: (7000, 29)
----------------------------------------
Após da limpeza grosseira: (7000, 13)
----------------------------------------


,count,mean,std,min,25%,50%,75%,max
vlrdocumento,7000.0,52.255201,59.783559,0.01,12.9875,33.980,70.695,1170.79
vlrliquido,7000.0,51.996624,59.469404,0.01,12.9800,33.905,70.555,1170.79


In [4]:
dp.cardinalidade(df_gastos_parlamentares.select_dtypes(include=["object"]))

,Atributo,Cardinalidade,Valores
3,txtnumero,14,"[7009787, 6992781, 6984585, 6971935, 6971636, 6911066, 6900895, 6867342, 6860309, 6860198, 6858322, 6836040, 6764634, 6736164]"
1,sguf,27,"[ES, SP, PI, SE, BA, PE, PR, RS, MG, SC, RJ, AC, PB, PA, AM, MA, CE, GO, MS, RO, RN, TO, MT, RR, AL, DF, AP]"
0,sgpartido,30,"[DEM, PP, PDT, PODE, PSB, PT, REPUBLICANOS, CIDADANIA, PSOL, MDB, PCdoB, PSD, PL, PROS, PSDB, PSC, PTB, PSL, SOLIDARIEDADE, NOVO, AVANTE, PATRIOTA, PV, REDE, PRB, PPS, PR, PATRI, PHS, PPL]"
2,txnomeparlamentar,810,"[Norma Ayub, Fausto Pinato, Iracema Portella, Fábio Henrique, Bacelar, Felipe Carreras, Arlindo Chinaglia, Celso Russomanno, Gustavo Fruet, Rubens Bueno, Henrique Fontana, Pompeo de Mattos, Ivan Valente, Júlio Delgado, Rui Falcão, Angela Amin, Benedita da Silva, Leonardo Monteiro, Hermes Parcianello, Ricardo Barros, Renildo Calheiros, Sérgio Brito, Darcísio Perondi, Perpétua Almeida, Wellington Roberto, Bosco Costa, Alice Portugal, Daniel Almeida, Elcione Barbalho, José Priante, Átila Lins, Mário Heringer, Odair Cunha, Patrus Ananias, Reginaldo Lopes, Gastão Vieira, Carlos Sampaio, Gilberto Nascimento, Vicentinho, Leônidas Cristino, Roberto Pessoa, Júlio Cesar, Paes Landim, Nilson Pinto, Silas Câmara, João Campos, Rubens Otoni, Gonzaga Patriota, Vander Loubet, Giacobo, Maria do Rosário, Paulo Pimenta, Wolney Queiroz, Átila Lira, Damião Feliciano, André de Paula, Luciano Bivar, Claudio Cajado, José Rocha, Paulo Magalhães, Lincoln Portela, Aécio Neves, Eduardo Barbosa, Rodrigo Maia, Mauro Lopes, Luiza Erundina, Jandira Feghali, Maurício Dziedricki, Marcelo Freixo, Márcio Jerry, Chico D'Angelo, Dra. Soraya Manato, Pedro Augusto Bezerra, Santini, Leur Lomanto Júnior, Eduardo Bolsonaro, Fernando Monteiro, Stefano Aguiar, Hugo Leal, Cacá Leão, Lafayette de Andrada, Gleisi Hoffmann, Jaqueline Cassol, Benes Leocádio, Vinicius Carvalho, João Marcelo Souza, Gelson Azevedo, Darci de Matos, Adriano do Baldy, Pedro Paulo, Ronaldo Martins, Pedro Lucas Fernandes, Enio Verri, Lídice da Mata, André Figueiredo, Marcelo Moraes, Paulo Ramos, Edmilson Rodrigues, Cristiano Vale, Afonso Hamm, ...]"


In [5]:
df_gastos_parlamentares.drop(["txtnumero"], axis=1, inplace=True)

len(df_gastos_parlamentares.columns)

12

## Sumário
Como o objetivo deste notebook é aprender a lidar com o bokeh. Vou deixar os atributos mais simples.
Nesta etapa de "data cleaning" fizemos:
1. Remoção dos atributos nulos;
2. Remoção de atributos com cardinalidade igual ou inferior a 1; e
3. Vimos que só há dois atributos numéricos de ponto flotuante (vlrdocumento, vlrliquido).

Com isso, passamos de 29 atributos para 12.

# Perguntas preliminares
Toda análise é movida por perguntas. **De posse dessa fonte de informações (1 dataset), o que gostaríamos de saber?**

1. Quem gastou mais em 2019?
2. Qual partido gastou mais na média em 2019?
3. Qual partido gastou mais na média em 2019 por Estado?
4. Como foi a distribuição dos gastos dos partidos ao longo dos meses?
4. Como foi a distribuição dos gastos dos partidos por ao longo dos meses por Estado?

In [6]:
df_gastos_parlamentares.head().T

,0,1,2,3,4
codlegislatura,56,56,56,56,56
idecadastro,66179,66828,67138,68720,69871
nucarteiraparlamentar,282,355,113,175,184
nudeputadoid,3163,2917,2320,3240,2985
nulegislatura,2019,2019,2019,2019,2019
numano,2020,2020,2020,2020,2020
nummes,1,1,1,1,1
sgpartido,DEM,PP,PP,PDT,PODE
sguf,ES,SP,PI,SE,BA
txnomeparlamentar,Norma Ayub,Fausto Pinato,Iracema Portella,Fábio Henrique,Bacelar


In [66]:
df_gastos = df_gastos_parlamentares.copy()

df_gastos["nummes"] = df_gastos["nummes"].apply(lambda x: str(x) if x > 10 else str(0)+str(x))

df_gastos["anomes"] = df_gastos["numano"].apply(str)+df_gastos["nummes"]
df_gastos["anomes"] = df_gastos["anomes"].apply(int)

df_gastos["deputado_partido"] = df_gastos["txnomeparlamentar"]+"-"+df_gastos["sgpartido"]

df_gastos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7000 entries, 0 to 999
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   codlegislatura         7000 non-null   int64  
 1   idecadastro            7000 non-null   int64  
 2   nucarteiraparlamentar  7000 non-null   int64  
 3   nudeputadoid           7000 non-null   int64  
 4   nulegislatura          7000 non-null   int64  
 5   numano                 7000 non-null   int64  
 6   nummes                 7000 non-null   object 
 7   sgpartido              7000 non-null   object 
 8   sguf                   7000 non-null   object 
 9   txnomeparlamentar      7000 non-null   object 
 10  vlrdocumento           7000 non-null   float64
 11  vlrliquido             7000 non-null   float64
 12  anomes                 7000 non-null   int64  
 13  deputado_partido       7000 non-null   object 
dtypes: float64(2), int64(7), object(5)
memory usage: 820.3+ K

## Quem gastou mais em 2019?

In [68]:
df_gastos_2020 = df_gastos[(df_gastos.anomes>=201901)
                           & (df_gastos.anomes<202001)]

df_gastos_por_deputado = (df_gastos_2020.groupby(["deputado_partido"])
                          .agg({"vlrdocumento": "sum","vlrliquido": "sum"})
                          .rename(columns={"vlrdocumento": "vlrdocumento_em_2019"
                                           , "vlrliquido": "vlrliquido_em_2019"})
                          .reset_index())

df_gastos_por_deputado = df_gastos_por_deputado.sort_values(by=["vlrliquido_em_2019"], ascending=False).head(15)

In [69]:
cds_gastos_por_deputado = ColumnDataSource(df_gastos_por_deputado)

# Create a figure
fig = figure(y_axis_label="Nome do deputado", x_axis_label="Gasto em 2019",
             y_range=df_gastos_por_deputado.deputado_partido, x_range=(0, max(df_gastos_por_deputado.vlrliquido_em_2019)+100),
             plot_height=300, plot_width=600,
             tools=['ypan', 'reset', 'save'])

fig.toolbar.logo = None

# Configure hbar
fig.hbar(right="vlrliquido_em_2019", y='deputado_partido', source=cds_gastos_por_deputado, height=0.2)

# -----------------------------------------------------------------------------
# Lista de tuplas contendo (Nome para exibição, Dado para aparecer) quando você passar o cursor em cima do item.
# Para o gráfico pctFig
tooltips_fig = [("Nome do deputado", '@deputado_partido')
                , ('Valor líquido', "@vlrliquido_em_2019{0.00}")]

hover_glyph_fig = fig.hbar(right="vlrliquido_em_2019", y='deputado_partido', source=cds_gastos_por_deputado, height=0.5
                           , alpha=0
                           , hover_fill_color="white"
                           , hover_alpha=0.3)

# Add the HoverTool to the figure
fig.add_tools(HoverTool(tooltips=tooltips_fig, renderers=[hover_glyph_fig]))

# -----------------------------------------------------------------------------
output_notebook()

show(fig)


Loading BokehJS ...

## Qual partido gastou mais na média em 2019?

In [112]:
df_gastos_2020 = df_gastos[(df_gastos.anomes>=201901)
                           & (df_gastos.anomes<202001)]

df_gastos_por_partido = (df_gastos_2020.groupby(["deputado_partido", "sgpartido", "anomes"])
                          .agg({"vlrdocumento": "sum","vlrliquido": "sum"})
                          .rename(columns={"vlrdocumento": "vlrdocumento_em_2019"
                                           , "vlrliquido": "vlrliquido_em_2019"})
                          .reset_index())

df_gastos_por_partido = df_gastos_por_partido.sort_values(by=["vlrliquido_em_2019"], ascending=False)


# Agrupando por partido e tirando a média

df_gastos_por_partido = (df_gastos_por_partido.
                         groupby(["sgpartido"]).
                         agg({"deputado_partido":"nunique"
                              , "vlrdocumento_em_2019": "mean"
                              , "vlrliquido_em_2019": "mean"}).
                         rename(columns={"deputado_partido":"qtd_deputados"
                                         , "vlrdocumento_em_2019": "media_vlrdocumento_em_2019"
                                         , "vlrliquido_em_2019": "media_vlrliquido_em_2019"}).
                         reset_index())

df_gastos_por_partido = df_gastos_por_partido.sort_values(by="media_vlrliquido_em_2019", ascending=False)

In [113]:
cds_gastos_por_partido = ColumnDataSource(df_gastos_por_partido)

# Create a figure
fig = figure(y_axis_label="Nome do partido", x_axis_label="Gasto médio em 2019",
             y_range=df_gastos_por_partido.sgpartido, x_range=(0, max(df_gastos_por_partido.media_vlrliquido_em_2019)+10),
             plot_height=380, plot_width=600,
             tools=['ypan', 'reset', 'save'])

fig.toolbar.logo = None

# Configure hbar
fig.hbar(right="media_vlrliquido_em_2019", y='sgpartido', source=cds_gastos_por_partido, height=0.65)

# -----------------------------------------------------------------------------
# Lista de tuplas contendo (Nome para exibição, Dado para aparecer) quando você passar o cursor em cima do item.
# Para o gráfico pctFig
tooltips_fig = [("Nome do partido", '@sgpartido')
                , ("Quantidade de deputados", "@qtd_deputados")
                , ('Valor líquido médio', "@media_vlrliquido_em_2019{0.00}")]

hover_glyph_fig = fig.hbar(right="media_vlrliquido_em_2019", y='sgpartido', source=cds_gastos_por_partido, height=.75
                           , alpha=0
                           , hover_fill_color="white"
                           , hover_alpha=0.3)

# Add the HoverTool to the figure
fig.add_tools(HoverTool(tooltips=tooltips_fig, renderers=[hover_glyph_fig]))

# -----------------------------------------------------------------------------
output_notebook()

show(fig)

Loading BokehJS ...